In [1]:
# !pip install pennylane

In [2]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

In [3]:
dev = qml.device("default.qubit",wires=3)
dev

<default.qubit device (wires=3) at 0x7f6c96eee410>

In [4]:
def layer(layer_weights):
    #L1
    for wire in range(3):
      qml.RY(layer_weights[0][wire], wires=wire)

    qml.CZ(wires=[0,1])
    qml.CZ(wires=[1,2])
    qml.CZ(wires=[2,0])

    #L2
    for wire in range(3):
      qml.RY(layer_weights[0][wire+3], wires=wire)

    qml.CZ(wires=[0,1])
    qml.CZ(wires=[1,2])
    qml.CZ(wires=[2,0])

    #L3
    for wire in range(3):
      qml.RY(layer_weights[0][wire+6], wires=wire)

    qml.CZ(wires=[0,1])
    qml.CZ(wires=[1,2])
    qml.CZ(wires=[2,0])

    #L4
    for wire in range(3):
      qml.RY(layer_weights[0][wire+9], wires=wire)

    qml.CZ(wires=[0,1])
    qml.CZ(wires=[1,2])
    qml.CZ(wires=[2,0])


In [5]:
def state_preparation(x):
    qml.BasisState(x, wires=[0, 1, 2, 3])

In [6]:
def accuracy(labels, predictions):
  state0 = qml.math.dm_from_state_vector(labels)
  state1 = qml.math.dm_from_state_vector(predictions)
  return qml.math.fidelity(state0, state1)

In [7]:
@qml.qnode(dev,diff_method="backprop")
def circuit(weights):
  layer(weights)
  return qml.state()

In [8]:
def variational_classifier(weights, x):
    return np.real(circuit(weights))

In [9]:
x = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8])
weights = np.random.rand(1,12)
print(weights)
predictions_test = variational_classifier(weights, x)
print(predictions_test)
print(accuracy(x,predictions_test))

[[0.59720817 0.0928125  0.01852606 0.88186272 0.87539517 0.60303122
  0.57065354 0.87199152 0.62109346 0.90083963 0.93824798 0.48589319]]
[-0.50556072  0.3003272   0.39352774 -0.0598458   0.59023596 -0.12887788
 -0.35198987 -0.08295461]
7.506527136286992e-05


In [10]:
def F1_loss(a,Appro_a):
  #   Define the cost function
  F1=(a[0]**2)*np.log2((Appro_a[0]**2)/(a[0]**2))+(a[1]**2)*np.log2((Appro_a[1]**2)/(a[1]**2))+(a[2]**2)*np.log2((Appro_a[2]**2)/(a[2]**2));
  F2=(a[3]**2)*np.log2((Appro_a[3]**2)/(a[3]**2))+(a[4]**2)*np.log2((Appro_a[4]**2)/(a[4]**2))+(a[5]**2)*np.log2((Appro_a[5]**2)/(a[5]**2));
  F3=(a[6]**2)*np.log2((Appro_a[6]**2)/(a[6]**2))+(a[7]**2)*np.log2((Appro_a[7]**2)/(a[7]**2));

  #   Prepare the Bell state
  Bell_State=np.ones((8,1))/np.linalg.norm(np.ones((8,1)));
  #   The sum of vector a
  Sum_a=np.sum(a);

  UaFunction=np.abs(Sum_a - np.linalg.norm(np.ones((8,1))) * np.transpose(Bell_State) *Appro_a)-(F1+F2+F3);

  return np.mean(np.real(UaFunction))

In [11]:
def cost(weights, X):
    predictions = variational_classifier(weights, X)
    return F1_loss(X, predictions)

In [12]:
opt = NesterovMomentumOptimizer(0.5)
np.random.seed(0)

In [13]:
# delta=1;
# Omega=1;
# gamma=1;
# Delta_t=0.01;

# alpha=np.sqrt(2)/2*delta*Delta_t;
# beta=(1-gamma*Delta_t/2);
# aa=[Delta_t*np.sqrt(2)/2*delta,Delta_t*gamma/4,np.sqrt(alpha**2+beta**2),Delta_t*gamma/4,Delta_t*gamma/4,Delta_t*gamma/4,Delta_t*gamma/4,Delta_t*gamma/4]
# x=np.sqrt(aa);
# norma=np.linalg.norm(x);
# x=x/np.linalg.norm(x);

# print(x)

In [14]:
weights = np.random.rand(1,12)
print(weights)
x = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8])

for it in range(100):
    # Update the weights by one optimizer step, using only a limited batch of data
    weights = opt.step(cost, weights, X=x)
    # Compute accuracy
    predictions = variational_classifier(weights, x)

    acc = accuracy(x, predictions)

    print(f"Iter: {it+1:4d} | Accuracy: {acc:0.7f}")
    print(weights)
    print("-----------------------------------------------------------")

[[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548  0.64589411
  0.43758721 0.891773   0.96366276 0.38344152 0.79172504 0.52889492]]
Iter:    1 | Accuracy: 0.0141653
[[ 3.3071251  -2.50699832 -2.3458855  -3.01180823  0.37637882  2.04946484
   0.77976274 -9.82182002 -7.76330274  5.67054368 11.44890212  9.31398796]]
-----------------------------------------------------------
Iter:    2 | Accuracy: 0.0533583
[[  6.88585661  -5.69316717  -4.75151512  -5.86258437   0.08821531
    3.43354437   0.34041566 -19.22472498 -15.39966452   9.82464753
   22.0510863   16.20190221]]
-----------------------------------------------------------
Iter:    3 | Accuracy: 0.0968123
[[  7.32186419  -4.6790644   -6.61162762 -10.94697964  -2.78252516
    7.7645934   -7.24446919 -34.74188828 -18.76425503  15.28494474
   36.58707786  15.38425075]]
-----------------------------------------------------------
Iter:    4 | Accuracy: 0.2541618
[[ 13.09875126  -1.99361651  -9.59935815 -12.48404651  -4.26156033
   11

In [15]:
weights = np.array([[-528.52733935,177.04777907,252.57575141,-956.36174753,-731.70801641,-148.12401958,-788.69277811,-415.09554786,252.39231417,-222.60168382,875.66556276,-74.18456888]])
# Iter:   89 | Accuracy: 0.7659465


predictions_test = variational_classifier(weights, x)
print(predictions_test)
print(x)

[ 0.05739969  0.3601235   0.30863865  0.60269076 -0.05151379  0.62388185
  0.01866851  0.12763866]
[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8]
